In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor, as_completed

from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, LSTM, BatchNormalization, Dropout, Dense, Add, Flatten

from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

2024-07-31 23:38:17.414164: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 23:38:17.465571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 23:38:17.465614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 23:38:17.465645: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 23:38:17.474958: I tensorflow/core/platform/cpu_feature_g

# Load csv from Desktop

In [2]:
NUM_CLASSES = 2
CLASSES = np.array(['Legitimate', 'Suspicious'])
DATASET_DIR = "./"
VECTOR_LENGTH = 1 * 816

def csvToVector(file_path):
    data = pd.read_csv(file_path, header=None)
    vector = data.values.flatten()
    return vector

def process_file(class_idx, file_path):
    vector = csvToVector(file_path)
    return (vector, class_idx)

def load_data(dataset_dir):
    X = []
    y = []
    subdirs = ['benign_cms1', 'malware_cms1']
    futures = []

    with ThreadPoolExecutor() as executor:
        for class_idx, class_name in enumerate(subdirs):
            class_dir = os.path.join(dataset_dir, class_name)
            for file_name in os.listdir(class_dir):
                if file_name.endswith('.csv'):
                    file_path = os.path.join(class_dir, file_name)
                    futures.append(executor.submit(process_file, class_idx, file_path))

        for future in as_completed(futures):
            vector, class_idx = future.result()
            X.append(vector)
            y.append(class_idx)

    X = np.array(X)
    y = np.array(y)
    return X, y

In [3]:
X, y = load_data(DATASET_DIR)

In [4]:
print(X.shape)
print(y.shape)
print(X)
print(y)

(4020, 816)
(4020,)
[[  103     0     0 ...  8838     0    29]
 [  223     0     0 ... 40174     0   274]
 [  298     0     0 ...  8843     0    12]
 ...
 [  145     0     0 ... 34875     0    53]
 [  186     0     0 ...  2669     0    57]
 [  148     0     0 ...  2669     0    54]]
[0 0 0 ... 1 1 1]


# Train, Validation, Test Split and Nomalize

In [34]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=42)

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train / 299.0
#X_val = X_val / 299.0
X_test = X_test / 299.0

y_train = to_categorical(y_train, 2)
#y_val = to_categorical(y_val, 2)
y_test = to_categorical(y_test, 2)

In [35]:
print(X_train.shape)
print(X_test.shape)
#print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
#print(y_val.shape)

(2814, 816)
(1206, 816)
(2814, 2)
(1206, 2)


# 1D CNN Architecture

In [36]:
input_layer = Input(shape=(VECTOR_LENGTH, 1))

x = LSTM(32, return_sequences=True)(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = LSTM(64, return_sequences=True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = LSTM(128, return_sequences=False)(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Flatten()(x)

x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 816, 1)]          0         
                                                                 
 lstm_9 (LSTM)               (None, 816, 32)           4352      
                                                                 
 batch_normalization_12 (Ba  (None, 816, 32)           128       
 tchNormalization)                                               
                                                                 
 dropout_12 (Dropout)        (None, 816, 32)           0         
                                                                 
 lstm_10 (LSTM)              (None, 816, 64)           24832     
                                                                 
 batch_normalization_13 (Ba  (None, 816, 64)           256       
 tchNormalization)                                         

# CheckPoint

In [61]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)
checkpoint = ModelCheckpoint(
    filepath='/tmp/CMS1_LSTM_CheckPoint.h5',
    save_best_only=True,
    monitor='accuracy',
    mode='max',
    verbose=1
)

# Model Training

In [86]:
model.fit(X_train, y_train, epochs=30, batch_size=32, callbacks=[reduce_lr, checkpoint])

Epoch 1/30
88/88 [==============================] - ETA: 0s - loss: 0.1199 - accuracy: 0.9684
Epoch 1: accuracy did not improve from 0.96979
88/88 [==============================] - 7s 77ms/step - loss: 0.1199 - accuracy: 0.9684 - lr: 2.5000e-04
Epoch 2/30
88/88 [==============================] - ETA: 0s - loss: 0.1127 - accuracy: 0.9680
Epoch 2: accuracy did not improve from 0.96979
88/88 [==============================] - 7s 77ms/step - loss: 0.1127 - accuracy: 0.9680 - lr: 2.5000e-04
Epoch 3/30
88/88 [==============================] - ETA: 0s - loss: 0.1210 - accuracy: 0.9705
Epoch 3: accuracy improved from 0.96979 to 0.97050, saving model to /tmp/CMS1_LSTM_CheckPoint.h5
88/88 [==============================] - 7s 78ms/step - loss: 0.1210 - accuracy: 0.9705 - lr: 2.5000e-04
Epoch 4/30
 1/88 [..............................] - ETA: 6s - loss: 0.1937 - accuracy: 0.9375

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


88/88 [==============================] - ETA: 0s - loss: 0.1103 - accuracy: 0.9698
Epoch 4: accuracy did not improve from 0.97050
88/88 [==============================] - 7s 77ms/step - loss: 0.1103 - accuracy: 0.9698 - lr: 2.5000e-04
Epoch 5/30
88/88 [==============================] - ETA: 0s - loss: 0.1078 - accuracy: 0.9701
Epoch 5: accuracy did not improve from 0.97050
88/88 [==============================] - 7s 77ms/step - loss: 0.1078 - accuracy: 0.9701 - lr: 2.5000e-04
Epoch 6/30
88/88 [==============================] - ETA: 0s - loss: 0.1070 - accuracy: 0.9705
Epoch 6: accuracy did not improve from 0.97050
88/88 [==============================] - 7s 77ms/step - loss: 0.1070 - accuracy: 0.9705 - lr: 2.5000e-04
Epoch 7/30
88/88 [==============================] - ETA: 0s - loss: 0.1160 - accuracy: 0.9680
Epoch 7: accuracy did not improve from 0.97050
88/88 [==============================] - 7s 78ms/step - loss: 0.1160 - accuracy: 0.9680 - lr: 2.5000e-04
Epoch 8/30
88/88 [=========

# Load Best CheckPoint

In [123]:
cp_model = load_model('/tmp/re10.h5')
cp_model.evaluate(X_test, y_test, batch_size=1000)

2/2 [==============================] - 1s 41ms/step - loss: 0.1195 - accuracy: 0.9652


[0.11954624950885773, 0.96517413854599]

# Evaluate

In [124]:
y_pred = cp_model.predict(X_test)

38/38 [==============================] - 2s 30ms/step


In [125]:
preds_single = CLASSES[np.argmax(y_pred, axis = -1)]
actual_single = CLASSES[np.argmax(y_test, axis = -1)]

# Classification Results

In [126]:
y_pred_classes = tf.argmax(y_pred, axis=1)
y_test_classes = tf.argmax(y_test, axis=1)

print(classification_report(y_test_classes, y_pred_classes, digits=4))

              precision    recall  f1-score   support

           0     0.9462    0.9888    0.9670       623
           1     0.9874    0.9400    0.9631       583

    accuracy                         0.9652      1206
   macro avg     0.9668    0.9644    0.9651      1206
weighted avg     0.9661    0.9652    0.9651      1206



# Confusion MatriX

In [81]:
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

class_labels = ['Legitimate', 'Suspicious']

conf_matrix_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)

print("Confusion Matrix:")
print(conf_matrix_df)

Confusion Matrix:
            Legitimate  Suspicious
Legitimate         615           8
Suspicious          41         542


### 